# Commits extraction and analysis

Import and install libraries

In [53]:
!pip install PyGithub

In [54]:
from github import Github, ApplicationOAuth
import pandas as pd

In [70]:
def key_with_extremum_value(d, type_of_extr):
    v=list(d.values())
    k=list(d.keys())
    if type_of_extr == 'max':
        return k[v.index(max(v))]
    else:
        return k[v.index(min(v))]
    
def get_latest_commits_from_repo(g, repo_name, count_of_commits = 50):
    repo = g.get_repo(repo_name)
    commits = repo.get_commits()
    commits = commits[0:count_of_commits - 1]
    return commits

def get_top_committers(commits):
    top_committers = {}
    for commit in commits:
        if commit.committer is not None:
            if top_committers.get(commit.committer.login) is None:
                top_committers[commit.committer.login] = 0
            else:
                top_committers[commit.committer.login] += 1

    top_committers = sorted(top_committers.items(), key=lambda x: x[1], reverse=True)
    
    print("Top committers: ")
    print(top_committers)
    print()
    return top_committers

def get_extr_line_changers(commits):
    top_strings_changed = {}
    for commit in commits:
        count = 0
        if commit is not None:
          for file in commit.files:
              count += file.changes
          if commit.committer is not None:
              if top_strings_changed.get(commit.committer.login) is None:
                  top_strings_changed[commit.committer.login] = count
              else: 
                  top_strings_changed[commit.committer.login] += count
    max_lines_changer = key_with_extremum_value(top_strings_changed, 'max')
    min_lines_changer = key_with_extremum_value(top_strings_changed, 'min')
    print("Person with max count of changed lines: " + max_lines_changer + " : " + str(top_strings_changed[max_lines_changer] ))
    print("Person with min count of changed lines: " + min_lines_changer + " : " + str(top_strings_changed[min_lines_changer] ))
    print()

def get_top_files_changed(commits):
    top_files_changed = {}
    for commit in commits:
        count = 0
        if commit is not None:
            for file in commit.files:
                if top_files_changed.get(file.filename) is None:
                    top_files_changed[file.filename] = 0
                else: 
                    top_files_changed[file.filename] += 1

    top_files_changed = sorted(top_files_changed.items(), key=lambda x: x[1], reverse=True)
    print("Top files changed with number of changes: ")
    for k, v in top_files_changed:
        print(k + " : " + str(v))
    print()

def create_df(commits):
  df = pd.DataFrame(columns = ["Login of committer", "FileName", "Changes"])
  list_of_series = []
  for commit in commits:
    for file in commit.files:
      if commit.committer is not None:
        list_of_series += [pd.Series([commit.committer.login, file.filename, file.patch], index=df.columns)]
  df = df.append(list_of_series , ignore_index=True)
  return df
  

In [71]:
def analyze_repo(username, password, repo_name, path_to_save_csv = "Last commits.csv", count_of_commits = 50):
    g = Github(username, password)
    commits = get_latest_commits_from_repo(g, repo_name, count_of_commits)
    get_top_committers(commits)
    get_extr_line_changers(commits)
    get_top_files_changed(commits)
    df = create_df(commits)
    csv=df.to_csv(path_to_save_csv)
    return df

### Here you have to input your parameters to receive result:

In [ ]:
USERNAME = "USERNAME"
PASSWORD = "PASSWORD"
NAME_REPO = "NAME_REPO"
PATH_TO_SAVE_CSV = "Last commits.csv"
COUNT_OF_COMMITS = 50
df = analyze_repo( USERNAME, PASSWORD,  NAME_REPO, PATH_TO_SAVE_CSV, COUNT_OF_COMMITS)
df.head()